<a href="https://colab.research.google.com/github/mohituoh/DeepLearning/blob/master/pretrained_vgg16_resnet_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn

In [5]:
transform_train=transforms.Compose( [transforms.RandomResizedCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)), 
])

transform_test=transforms.Compose( [transforms.RandomResizedCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),   
])

In [6]:
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform_train)
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
batch_size=16
trainloader=torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False)

In [8]:
vgg=models.vgg16_bn(pretrained=True)

In [9]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [10]:
for param in vgg.parameters():
  param.requires_grad=False

In [11]:
fianl_in_feature=vgg.classifier[6].in_features

In [12]:
mod_classifier=list(vgg.classifier[:-1])
mod_classifier.extend([nn.Linear(in_features=4096,out_features=10,bias=True)])

In [13]:
mod_classifier

[Linear(in_features=25088, out_features=4096, bias=True),
 ReLU(inplace=True),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=4096, bias=True),
 ReLU(inplace=True),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=10, bias=True)]

In [14]:
vgg.classifier=nn.Sequential(*mod_classifier)
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [15]:
for param in vgg.parameters():

  if param.requires_grad:
    print(param.shape)



torch.Size([10, 4096])
torch.Size([10])


In [16]:
device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")

In [17]:
print(device)

cuda:0


In [18]:
import torch.optim as optim

In [19]:
vgg=vgg.to(device)
loss_fn=nn.CrossEntropyLoss()
opt=optim.SGD(vgg.parameters(),lr=0.5)

In [20]:
def evaluation(dataloader,model):
  total,correct=0,0
  for data in dataloader:
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    outputs=model(inputs)
    _,pred=torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (pred==labels).sum().item()
    return 100*correct/total

In [21]:
import numpy as np
import matplotlib.pyplot as plt


In [22]:
loss_epoch_arr=[]
max_epoch=1
n_iters=np.ceil(50000/batch_size)
for epoch in range(max_epoch):
  for i , data in enumerate(trainloader,0):
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    opt.zero_grad()
    outputs=vgg(inputs)
    loss=loss_fn(outputs,labels)
    loss.backward()
    opt.step()
    del inputs,labels,outputs
    torch.cuda.empty_cache()

    if i%100==0:
      print("iteration %d %d  loss : %2f"%(i,n_iters,loss.item()))
  loss_epoch_arr.append(loss.item())
  print("epoch: %d %d,Test_acc: %0.2f ,Train_acc: %0.2f" %(epoch,max_epoch,evaluation(testloader,vgg),evaluation(trainloader,vgg)))
plt.plot(loss_epoch_arr)
plt.show()


iteration 0 3125  loss : 2.171194
iteration 100 3125  loss : 11.141139
iteration 200 3125  loss : 6.149672


KeyboardInterrupt: ignored

#with model *copies*

In [23]:
import copy

In [24]:
loss_epoch_arr=[]
max_epoch=1
min_loss=1000
n_iters=np.ceil(50000/batch_size)
for epoch in range(max_epoch):
  for i , data in enumerate(trainloader,0):
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    opt.zero_grad()
    outputs=vgg(inputs)
    loss=loss_fn(outputs,labels)
    loss.backward()
    opt.step()
    if min_loss>loss.item():
      min_loss=loss.item()
      best_model= copy.deepcopy(vgg.state_dict())
      print("Min loss :- %0.2f" % min_loss)


    del inputs,labels,outputs
    torch.cuda.empty_cache()

    if i%100==0:
      print("iteration %d %d  loss : %2f"%(i,n_iters,loss.item()))
  loss_epoch_arr.append(loss.item())
  print("epoch: %d %d,Test_acc: %0.2f ,Train_acc: %0.2f" %(epoch,max_epoch,evaluation(testloader,vgg),evaluation(trainloader,vgg)))
plt.plot(loss_epoch_arr)
plt.show()


Min loss :- 11.02
iteration 0 3125  loss : 11.020144
Min loss :- 10.99
Min loss :- 5.91


KeyboardInterrupt: ignored

In [25]:
vgg.load_state_dict(best_model)
print(evaluation(testloader,vgg),evaluation(trainloader,vgg))

25.0 18.75


#Resnet Model


In [26]:
resnet=models.resnet18(pretrained=True)

In [27]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
for param in resnet.parameters():
  param.requires_grad=False

In [29]:
in_features=resnet.fc.in_features
resnet.fc=nn.Linear(in_features=in_features,out_features=10,bias=True)

In [30]:
for param in resnet.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 512])
torch.Size([10])


In [31]:
resnet=resnet.to(device)
loss_fn=nn.CrossEntropyLoss()
opt=optim.SGD(resnet.parameters(),lr=0.01)

In [32]:
loss_epoch_arr=[]
max_epoch=4
min_loss=1000
n_iters=np.ceil(50000/batch_size)
for epoch in range(max_epoch):
  for i , data in enumerate(trainloader,0):
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    opt.zero_grad()
    outputs=vgg(inputs)
    loss=loss_fn(outputs,labels)
    loss.backward()
    opt.step()
    if min_loss>loss.item():
      min_loss=loss.item()
      best_model= copy.deepcopy(vgg.state_dict())
      print("Min loss :- %0.2f" % min_loss)


    del inputs,labels,outputs
    torch.cuda.empty_cache()

    if i%100==0:
      print("iteration %d %d  loss : %2f"%(i,n_iters,loss.item()))
  loss_epoch_arr.append(loss.item())
  print("epoch: %d %d,Test_acc: %0.2f ,Train_acc: %0.2f" %(epoch,max_epoch,evaluation(testloader,vgg),evaluation(trainloader,vgg)))
plt.plot(loss_epoch_arr)
plt.show()

Min loss :- 8.46
iteration 0 3125  loss : 8.455293
Min loss :- 5.40


KeyboardInterrupt: ignored

In [33]:
resnet.load_state_dict(best_model)
print(evaluation(testloader,vgg),evaluation(trainloader,vgg))

RuntimeError: ignored

#Inception Model

In [34]:
inception =models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [35]:
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [36]:
for param in inception.parameters():
   param.requires_grad=False


In [39]:
aux_in_features=inception.AuxLogits.fc.in_features
inception.AuxLogits.fc=nn.Linear(aux_in_features, out_features=10)

In [40]:
for param in inception.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])


In [42]:
in_features=inception.fc.in_features
inception.fc=nn.Linear(in_features,out_features=10)

In [43]:
for param in inception.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])
torch.Size([10, 2048])
torch.Size([10])


In [44]:
transform_train=transforms.Compose( [transforms.RandomResizedCrop(299),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)), 
])

transform_test=transforms.Compose( [transforms.RandomResizedCrop(299),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),   
])

In [45]:
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform_train)
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [46]:
batch_size=16
trainloader=torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False)

In [49]:
inception=inception.to(device)
loss_fn=nn.CrossEntropyLoss()
opt=optim.SGD(inception.parameters(),lr=0.01)

In [50]:
def evaluation_Inception(dataloader,model):
  total,correct=0,0
  for data in dataloader:
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    outputs,aux_outputs=model(inputs)
    _,pred=torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (pred==labels).sum().item()
    return 100*correct/total

In [52]:
loss_epoch_arr=[]
max_epoch=4
min_loss=1000
n_iters=np.ceil(50000/batch_size)
for epoch in range(max_epoch):
  for i , data in enumerate(trainloader,0):
    inputs,labels=data
    inputs,labels=inputs.to(device),labels.to(device)
    opt.zero_grad()
    outputs,aux_outputs=inception(inputs)
    loss=loss_fn(outputs,labels)+0.3*loss_fn(aux_outputs,labels)
    loss.backward()
    opt.step()
    if min_loss>loss.item():
      min_loss=loss.item()
      best_model= copy.deepcopy(inception.state_dict())
      print("Min loss :- %0.2f" % min_loss)


    del inputs,labels,outputs
    torch.cuda.empty_cache()

    if i%100==0:
      print("iteration %d %d  loss : %2f"%(i,n_iters,loss.item()))
  loss_epoch_arr.append(loss.item())
  print("epoch: %d %d,Test_acc: %0.2f ,Train_acc: %0.2f" %(epoch,max_epoch,evaluation_Inception(testloader,inception),evaluation_Inception(trainloader,inception)))
plt.plot(loss_epoch_arr)
plt.show()

Min loss :- 3.18
iteration 0 3125  loss : 3.179649
Min loss :- 3.02
Min loss :- 2.95
Min loss :- 2.84
Min loss :- 2.82
Min loss :- 2.78
Min loss :- 2.59
Min loss :- 2.58
iteration 100 3125  loss : 2.710702
Min loss :- 2.52
Min loss :- 2.49
Min loss :- 2.40
Min loss :- 2.32
Min loss :- 2.27
Min loss :- 2.25
Min loss :- 2.08
iteration 200 3125  loss : 2.389116
Min loss :- 1.97
iteration 300 3125  loss : 2.732201
Min loss :- 1.96
Min loss :- 1.74
Min loss :- 1.67
iteration 400 3125  loss : 2.332165
iteration 500 3125  loss : 2.517910
Min loss :- 1.59
iteration 600 3125  loss : 2.213941
Min loss :- 1.44
iteration 700 3125  loss : 1.667660
Min loss :- 1.38
iteration 800 3125  loss : 1.806150
iteration 900 3125  loss : 2.141164
iteration 1000 3125  loss : 1.527117
iteration 1100 3125  loss : 1.987516
iteration 1200 3125  loss : 2.124191
iteration 1300 3125  loss : 2.084512
Min loss :- 1.32
iteration 1400 3125  loss : 2.259082
iteration 1500 3125  loss : 1.666375
iteration 1600 3125  loss : 1

KeyboardInterrupt: ignored